## Content-Based Filtering

### Setup

Importing Libraries

In [1]:
import gradio as gr
import numpy as np
import os
import pandas as pd
import pickle
import torch

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

Loading Preprocessed Data

In [2]:
df_merged = pd.read_csv("preprocessed_data/merged_metadata_keywords.csv")
df_qualified = pd.read_csv("preprocessed_data/qualified_movies.csv")

### Sentence Embedding Generation and Similarity Calculation

- Initialize the model with the `all-MiniLM-L6-v2` pre-trained model
- Encode a list of sentences in the `soup` column using the pre-trained model

In [3]:
# Uncomment this cell if you run it for the first time

# if torch.cuda.is_available():
#     device = "cuda"
# else:
#     device = "cpu"
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2").to(device)

# sentence_embeddings = model.encode(df["soup"].tolist())

# folder_name = "embedding_data_cbf"

# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)

# with open(f"{folder_name}\sentence_embeddings.pkl", "wb") as f:
#     pickle.dump(sentence_embeddings, f)

In [4]:
folder_name = "embedding_data_cbf"

# Import sentence_embeddings from the file
with open(f"{folder_name}\sentence_embeddings.pkl", "rb") as f:
    sentence_embeddings = pickle.load(f)

sentence_embeddings.shape

(44064, 384)

Compute cosine similarity

In [5]:
# Uncomment this cell if you run it for the first time

# cos_sim = cosine_similarity(sentence_embeddings)

# with open(f"{folder_name}\cos_sim.pkl", "wb") as f:
#     pickle.dump(cos_sim, f)

In [6]:
# Import cos_sim from the file
with open(f"{folder_name}\cos_sim.pkl", "rb") as f:
    cos_sim = pickle.load(f)

cos_sim.shape

(44064, 44064)

Construct a reverse map of movie titles to indices

In [7]:
movie_indices = pd.Series(
    df_merged.index, index=df_merged["title"].apply(lambda title: title.lower())
).drop_duplicates()
movie_indices.head()

title
toy story                      0
jumanji                        1
grumpier old men               2
waiting to exhale              3
father of the bride part ii    4
dtype: int64

### Pipeline
Steps:
1. Get a sorted DataFrame of movies based on their similarity scores to a given movie.
2. Filter out movies that are not in the qualified movies chart and sort the movies based on similarity scores and IMDB's weighted rating.
3. Get 5 movie recommendations based on a given title using content-based filtering.

In [8]:
def get_sorted_similar_movies(
    title: str, cos_sim: np.ndarray, df_merged: pd.DataFrame
) -> list[int]:
    """
    Get a sorted DataFrame of movies based on their similarity scores to a given movie.

    :param title: The title of the movie to find similar movies for.
    :param cos_sim: The cosine similarity matrix of movies.
    :param df_merged: The DataFrame containing movie details.
    :return: A sorted DataFrame of similar movies.
    """
    try:
        # Get the index of the movie that matches the title
        movie_index = movie_indices[title.lower()]

        # If there are multiple movies with the same title, pick the first one.
        if isinstance(movie_index, pd.Series):
            movie_index = movie_index[0]

    except KeyError:
        print(f"Movie '{title}' not found. Please enter a valid movie title.")
        return None

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cos_sim[movie_index]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:]

    # Get the movie indices
    sorted_movie_indices = [sim_score[0] for sim_score in sim_scores]

    # Get the similarity scores
    sorted_similarity_scores = [format(sim_score[1], ".1f") for sim_score in sim_scores]

    movie_details = [
        "id",
        "title",
        "genres",
        "original_language",
        "production_countries",
        "release_date",
        "runtime",
        "weighted_rating",
    ]

    sorted_similar_movies = df_merged.loc[sorted_movie_indices, movie_details]

    sorted_similar_movies["similarity_scores"] = sorted_similarity_scores

    return sorted_similar_movies

In [9]:
def get_qualified_movies(
    df_qualified: pd.DataFrame, sorted_similar_movies: pd.DataFrame
) -> pd.DataFrame:
    """
    Filter out movies that are not in the qualified movies chart and sort the movies based on similarity scores and IMDB's weighted rating.

    :param df_qualified: The DataFrame containing qualified movie details.
    :param sorted_similar_movies: The DataFrame containing movie details sorted by similarity scores.
    :return: A Pandas DataFrame containing the qualified movies sorted by similarity scores and IMDB's weighted rating..
    """

    qualified_movies = sorted_similar_movies[
        sorted_similar_movies["id"].isin(df_qualified["id"])
    ]
    qualified_movies = qualified_movies.sort_values(
        by=["similarity_scores", "weighted_rating"], ascending=False
    )
    return qualified_movies

In [10]:
def get_movie_recommendations_cbf(title: str) -> pd.DataFrame:
    """
    Get movie recommendations based on a given title using content-based filtering.

    :param title: The title of the movie to find similar movies for.
    :return: A Pandas DataFrame containing the recommended movies
    """
    # Get recommended movie indices based on the given title
    sorted_similar_movies = get_sorted_similar_movies(title, cos_sim, df_merged)

    # Filter out bad movies
    qualified_movies = get_qualified_movies(df_qualified, sorted_similar_movies)

    recommended_movies = qualified_movies.head(5)
    recommended_movies.columns = [
        "ID",
        "Title",
        "Genres",
        "Language",
        "Production Countries",
        "Release Date",
        "Runtime",
        "Weighted Rating",
        "Similarity Score",
    ]

    recommendation_criteria = recommended_movies[["ID", "Title", "Similarity Score", "Weighted Rating"]]
    recommended_movies.drop(["Similarity Score", "Weighted Rating"], axis = 1, inplace=True)
    return recommended_movies, recommendation_criteria

### Demo

In [11]:
with gr.Blocks(theme=gr.themes.Soft(text_size="lg")) as demo:
    gr.Markdown(
        """
    # Movie Recommendation System
    
    """
    )
    title = gr.Dropdown(
        choices=df_merged["title"].unique().tolist(),
        label="Movie Title",
        value="Iron Man",
    )
    recommend_button = gr.Button("Get Movie Recommendations")
    recommended_movies = gr.DataFrame(label="Movie Recommendations")
    recommendation_criteria = gr.DataFrame(label="Recommendation Criteria")
    recommend_button.click(
        get_movie_recommendations_cbf,
        inputs=[title],
        outputs=[recommended_movies, recommendation_criteria],
    )
    examples = gr.Examples(
        examples=[
            "Captain America: The First Avenger",
            "The Conjuring",
            "Toy Story",
            "Final Destination 5",
        ],
        inputs=[title],
    )

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\garyt\AppData\Local\Temp\ipykernel_19168\3452235507.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_movies.drop(["Similarity Score", "Weighted Rating"], axis = 1, inplace=True)
C:\Users\garyt\AppData\Local\Temp\ipykernel_19168\3452235507.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_movies.drop(["Similarity Score", "Weighted Rating"], axis = 1, inplace=True)
